In [26]:
import dlib
import face_recognition
import cv2
import os,sys
import numpy as np
import math

In [27]:
def face_confidence(face_distance,face_match_threshold = 0.6) :
    range_val = (1-face_match_threshold)
    linear_val = (1-face_distance)/(range_val * 2)
    if face_distance > face_match_threshold :
        return str(round(linear_val * 100,2)) + '%'
    else :
        val = (linear_val + ((1-linear_val) * math.pow((linear_val-0.5)*2,0.2))) * 100
        return str(round(val,2)) + '%'

In [28]:
class FaceRec :
    face_locations = []
    face_encodings = []
    face_names = []
    known_face_encodings = []
    known_face_names = []
    process_current_frame = True
    
    def __init__(self) :
        self.encode_faces()
        
    def encode_faces(self) :
        self.known_face_encodings = []
        self.known_face_names = []
        for image in os.listdir('faces'):
            face_image = face_recognition.load_image_file(f'faces/{image}')
            face_encoding = face_recognition.face_encodings(face_image)[0]
            self.known_face_encodings.append(face_encoding)
            self.known_face_names.append(image)
        print(self.known_face_names)
    
    def recognise(self) :
        video_capture = cv2.VideoCapture(0)
        while True :
            ret,frame  = video_capture.read()
        
            imgS = cv2.resize(frame,(0,0),None,0.25,0.25)
            imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

            self.face_locations = face_recognition.face_locations(imgS)
            self.face_encodings = face_recognition.face_encodings(imgS,self.face_locations)
            self.face_names = []
            for face_encoding in self.face_encodings :
                matches = face_recognition.compare_faces(self.known_face_encodings,face_encoding)
                name = 'unknown'
                confidence = 'unknown'

                face_distances = face_recognition.face_distance(self.known_face_encodings,face_encoding)
                best_match_index = np.argmin(face_distances)

                if matches[best_match_index] :
                    name = self.known_face_names[best_match_index]
                    confidence = face_confidence(face_distances[best_match_index])
                self.face_names.append(f'{name} ({confidence})')
               
            
            #display
            for (t,r,b,l) , name in zip(self.face_locations,self.face_names) :
                t,r,b,l = t * 4,r * 4,b * 4,l * 4 #resize to orignal shape
                #print(t,r,b,l,name)
                if name != 'unknown (unknown)' :
                    cv2.rectangle(frame,(l,t),(r,b),(0,255,0),2)
                    cv2.rectangle(frame,(l,b-35),(r,b),(0,255,0),-1)
                else :
                    cv2.rectangle(frame,(l,t),(r,b),(0,0,255),2)
                    cv2.rectangle(frame,(l,b-35),(r,b),(0,0,255),-1)
                
                cv2.putText(frame,name,(l + 6,b - 6),cv2.FONT_HERSHEY_DUPLEX,0.8,(255,255,255),1)
            cv2.imshow('Face Recognition',frame)
            if cv2.waitKey(1)& 0xFF == ord('q') :
                break
                
        video_capture.release()
        cv2.destroyAllWindows()

In [29]:
fr = FaceRec()

['Aditya Kalhan.jpg', 'Elon Musk.jpg', 'harry styles.jpg', 'MESSI.jpg', 'Ryan Reynolds.jpg']


In [30]:
fr.recognise()